In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import os
#os.chdir("/content/drive/MyDrive/DL_Project/Experiment3")
#!ls

Exp3.ipynb  test_dataset  train_dataset  val_dataset


In [ ]:
import gdown

dataset = 'https://drive.google.com/uc?id=18Q_YSek5Zgs1kXl68MPQJIvyo4KepJvj&export=download'
output='./new.zip'
gdown.download(dataset, output, quiet=False)

In [ ]:
import zipfile

with zipfile.ZipFile('./new.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [12]:
#SCRIPT3 FIND AND REMOVE CORRUPTED IMAGES
import os
from os import listdir
from PIL import Image
import cv2

categ = ['train_dataset','test_dataset','val_dataset']
dataset = '/content/drive/MyDrive/DL_Project/Experiment3'
bad_file_list=[]
bad_count=0

for cat in categ:
  img_path = os.path.join(dataset, cat)
  for foldername in os.listdir(img_path):
    sign_path = os.path.join(img_path, foldername)
    print(sign_path)
    for sign in listdir(sign_path):
      if sign.endswith('.jpg'):
        try:
          Image.open(os.path.join(sign_path, sign)).load() # open the image file
           # verify that it is, in fact an image
        except:
          bad_file_list.append(os.path.join(sign_path, sign))
          bad_count +=1
print(bad_file_list)

/content/drive/MyDrive/DL_Project/Experiment3/train_dataset/2
/content/drive/MyDrive/DL_Project/Experiment3/train_dataset/3
/content/drive/MyDrive/DL_Project/Experiment3/train_dataset/1
/content/drive/MyDrive/DL_Project/Experiment3/test_dataset/2
/content/drive/MyDrive/DL_Project/Experiment3/test_dataset/1
/content/drive/MyDrive/DL_Project/Experiment3/test_dataset/3
/content/drive/MyDrive/DL_Project/Experiment3/val_dataset/3
/content/drive/MyDrive/DL_Project/Experiment3/val_dataset/2
/content/drive/MyDrive/DL_Project/Experiment3/val_dataset/1
[]


In [11]:
!rm '/content/drive/MyDrive/DL_Project/Experiment3/val_dataset/2/0c416a95-6e96-4e8c-bab3-56b682feafe9.jpg.jpg.jpg'

In [3]:
#SCRIPT4  CREATE DICTIONARY WITH CANDIDATE HYPERPARAMS

models = []
one = {"kernel_num": 64, "kernel_size": 3, "fc_size": 1024, "conv_layer":4}
two = {"kernel_num": 64, "kernel_size": 3, "fc_size": 1024, "conv_layer":3}
three = {"kernel_num": 32, "kernel_size": 3, "fc_size": 768, "conv_layer":3}
four = {"kernel_num": 32, "kernel_size": 3, "fc_size": 1024, "conv_layer":4}
five = {"kernel_num": 64, "kernel_size": 4, "fc_size": 512, "conv_layer":4}
six = {"kernel_num": 32, "kernel_size": 4, "fc_size": 768, "conv_layer":2}

models.append(one)
models.append(two)
models.append(three)
models.append(four)
models.append(five)
models.append(six)

In [4]:
#SCRIPT5 DEFINE SAVE/LOAD STATE
import os
import pickle
def save_state(mod,epochs,model_type):
    with open('state.state', 'wb') as cfg:
      pickle.dump(mod, cfg)
      pickle.dump(epochs,cfg)
      pickle.dump(model_type,cfg)

def load_state():
    with open('state.state','rb') as cffile: 
      model2 = pickle.load(cffile)
      epochs2 = pickle.load(cffile)
      model_type = pickle.load(cffile)
    return model2, epochs2, model_type

In [5]:
#create model function
from tensorflow.keras.layers import Input, Conv2D , Dropout, MaxPool2D, Flatten, Dense 
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import matplotlib.pyplot as plt
import sys
from tensorflow.keras.callbacks import CSVLogger

def create_model(ker_num,ker_size,fc_size,conv_layer_num):
  
  input = Input(shape =(150,150,3))
  weight_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=1234)
  bias_initializer=tf.keras.initializers.Zeros()

  if conv_layer_num == 2:
    x = Conv2D (filters =ker_num, kernel_size =ker_size, padding ='same', activation='relu',kernel_initializer=weight_initializer,kernel_regularizer=l2(0.00005),bias_initializer=bias_initializer)(input)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    x = Conv2D (filters =2*ker_num, kernel_size =ker_size, padding ='same', activation='relu',kernel_initializer=weight_initializer,kernel_regularizer=l2(0.00005),bias_initializer=bias_initializer)(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
  elif conv_layer_num == 3:

    x = Conv2D (filters =ker_num, kernel_size =ker_size, padding ='same', activation='relu',kernel_initializer=weight_initializer,kernel_regularizer=l2(0.00005),bias_initializer=bias_initializer)(input)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    x = Conv2D (filters =2*ker_num, kernel_size =ker_size, padding ='same', activation='relu',kernel_initializer=weight_initializer,kernel_regularizer=l2(0.00005),bias_initializer=bias_initializer)(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    x = Conv2D (filters =4*ker_num, kernel_size =ker_size, padding ='same', activation='relu',kernel_initializer=weight_initializer,kernel_regularizer=l2(0.00005),bias_initializer=bias_initializer)(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
  else:
    x = Conv2D (filters =ker_num, kernel_size =ker_size, padding ='same', activation='relu',kernel_initializer=weight_initializer,kernel_regularizer=l2(0.00005),bias_initializer=bias_initializer)(input)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    x = Conv2D (filters =2*ker_num, kernel_size =ker_size, padding ='same', activation='relu',kernel_initializer=weight_initializer,kernel_regularizer=l2(0.00005),bias_initializer=bias_initializer)(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    x = Conv2D (filters =4*ker_num, kernel_size =ker_size, padding ='same', activation='relu',kernel_initializer=weight_initializer,kernel_regularizer=l2(0.00005),bias_initializer=bias_initializer)(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    x = Conv2D (filters =8*ker_num, kernel_size =ker_size, padding ='same', activation='relu',kernel_initializer=weight_initializer,kernel_regularizer=l2(0.00005),bias_initializer=bias_initializer)(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)


  x = Flatten()(x)
  x = Dropout(0.3)(x)
  x = Dense(units = fc_size, activation ='relu', kernel_initializer=weight_initializer,kernel_regularizer=l2(0.00005),bias_initializer=bias_initializer)(x)
  output = Dense(units = 3, activation ='softmax')(x)

  model = Model (inputs=input, outputs =output)
  model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])


  return model


In [6]:
#FUNCTION FOR GETTING DATASETS
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_datasets():
#appply data augmentation on TRAIN DATA ONLY!!
  train_datagen = ImageDataGenerator( rescale = 1.0/255.,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True) 

  train_generator = train_datagen.flow_from_directory(
    directory=r"./train_dataset/",
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42)

  val_datagen = ImageDataGenerator( rescale = 1.0/255.)

  val_generator = val_datagen.flow_from_directory(
    directory=r"./val_dataset/",
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42)

  test_datagen = ImageDataGenerator( rescale = 1.0/255.)

  test_generator = test_datagen.flow_from_directory(
    directory=r"./test_dataset/",
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42)
  
  return train_generator, val_generator, test_generator
  #one, two, three = get_datasets()
  

In [7]:
#CALLBACK
import tensorflow as tf
import keras
from tensorflow.keras.callbacks import Callback
from timeit import default_timer as timer
stop_flag = False


class newcb(keras.callbacks.Callback):
  def __init__(self, start):
    self.start = start
    print(self.start,'\nINIT TIME\n')

  def on_epoch_end(self, epoch, logs=None):
      end = timer()
      print('\nCALLBACK time ', end,'\n')
      if end-self.start > 80000:
        print('\nTERMINATING TRAIN DUE TO EXCEEDING 5 HOURS\n')
        print(current_arch,' SAVING ARCH\n')
        save_state(self.model,epoch,current_arch)
        self.model.stop_training = True
        global stop_flag
        stop_flag = True
        print(stop_flag,' STOP FLAG CALLBACK\n')




In [8]:
#SCRIPT 9 CREATE CLASS WEIGHTS
import os

def compute_weigths():
  size_per_class = []
  a = os.listdir('./train_dataset/')
  b = [int(w) for w in a]
  b.sort()
  print(b)

  for i in b:
    temp = os.listdir('./train_dataset/'+ str(i))
    size_per_class.append(len(temp))

  #print(size_per_class)
  total = sum(size_per_class)
  #print(total)

  class_weights = {0: 1, 1: 1, 2: 1}
  #print(class_weights)
  for i in range(len(size_per_class)):
    weigth = (total)/(3*size_per_class[i])
    class_weights[i] = weigth

  #print(class_weights)
  return class_weights 

weights_per_class = compute_weigths()
print(weights_per_class)

[1, 2, 3]
{0: 0.4398543842988287, 1: 2.790160642570281, 2: 2.716520039100684}


In [ ]:
#MAIN TRAINNING SCRIPT
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback
import os
import matplotlib.pyplot as plt
import sys
import time

stop_flag = False
#MARK START TIME
start = timer()

#Initialize only for first run
epochs = 50
epochs_done = 0
current_arch = 0
current_model = None

#IF CONTINUING LOAD STATE
if os.path.exists('state.state'):
  print('\nINIT MODEL FROM SAVED STATE \n')
  current_model, epochs_done, current_arch = load_state()
  print(current_model.loss)
  print(current_model.optimizer)
  print(current_model.metrics_names)
  print(current_model.metrics)
#ELSE CREATE A MODEL  
else:
  print('\nCREATING MODEL\n')
#HYPERPARAMS FOR THIS RUN
  model_params = models[current_arch]
  kernel_num = model_params['kernel_num']
  kernel_size = model_params['kernel_size']
  fc_size = model_params['fc_size']
  conv_num = model_params['conv_layer']
  current_model = create_model(ker_num=kernel_num,ker_size=kernel_size,fc_size=fc_size,conv_layer_num=conv_num)
  print(current_model.summary())  

train_generator, val_generator, test_generator = get_datasets() 
print(train_generator.class_indices)
print(val_generator.class_indices)
print(test_generator.class_indices)

#EARLY STOP CALLBACK
callbackES = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10,restore_best_weights=True)

#CALL BACK FOR SAVING AT 5 HOURS
stop_cb = newcb(start=start)

#ITERATE THROUGH CANDIDATE MODELS
for arch in range(current_arch,len(models)):
  current_arch = arch
  print('RUNNING MODEL ',current_arch)
  if stop_flag == True:
    print('BREAKING FROM LOOP BECAUSE CALLBACK STOPPED TRAINNNG')
    break
#ITEREATE THROUGH EPOCHS  
  history=current_model.fit(train_generator,validation_data = val_generator,epochs = epochs-epochs_done,class_weight=weights_per_class,verbose = 1,shuffle = True,callbacks=[stop_cb,callbackES])

  fig, ax = plt.subplots(2)

  ax[0].set_ylabel('Loss')
  ax[1].set_ylabel('Accuracy')

  ax[0].plot(history.history['loss'], color='red') 
  ax[0].plot(history.history['val_loss'], color='blue') 

  ax[1].plot(history.history['accuracy'], color='red') 
  ax[1].plot(history.history['val_accuracy'], color='blue') 

  filename2 = 'train_history'+str(arch)+'.png'
  fig.savefig(filename2)
  plt.show()

  print(stop_flag,' AFTER FIT FLAG IS')
  file_name = 'model_' + str(arch) + '.model'
  
  if stop_flag == False:
   
   with open(file_name, 'wb') as cfg:
      pickle.dump(current_model,cfg)

   model_params = models[current_arch+1]
   kernel_num = model_params['kernel_num']
   kernel_size = model_params['kernel_size']
   fc_size = model_params['fc_size']
   conv_num = model_params['conv_layer']
   current_model = create_model(ker_num=kernel_num,ker_size=kernel_size,fc_size=fc_size,conv_layer_num=conv_num)
   print(current_model.summary())  
   epochs_done = 0   



CREATING MODEL

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 150, 150, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 38, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 38, 38, 